In [1]:
import json
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split

# Сформировать множество из разных файлов

In [2]:
df = pd.read_csv("../../Data/Raw/merged_with_labels_11_patterns.csv", sep=";")
print(len(df[df['x'].isna()]), "nan texts")
df = df[~df['x'].isna()]

df_3 = pd.read_csv("../../Data/Raw/12052022_one_cmd.csv", sep=";")

df_3.drop("delayed", axis=1, inplace=True)

df_4 = pd.read_csv("/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/Raw/hour_yandex.csv", sep=";")
df_4.drop("delayed", axis=1, inplace=True)
df_5 = pd.read_csv("/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/Raw/turn_around.csv", sep=";")
df_5.drop("delayed", axis=1, inplace=True)

0 nan texts


In [3]:
df = pd.concat([df_3, df[df["type"].isin(["students", "crowdsource"])], df_4, df_5])

In [4]:
df.drop_duplicates(inplace=True)
df.shape

(21535, 16)

# предобработка

In [5]:
# в случае изменения функции, изменить её и в обработке длинных команд в Analyze_complex_commands.ipynb
#punct_characters = re.escape(".,!?:;-#$*()[]'\"/|^%~`_+-")
import re
def removePunctETC(value):
    if type(value) is float:
        return value
    value = value.lower()
    value = re.sub("[^\w\s°]", " ", value)
    return value
df["x"] = df["x"].map(removePunctETC)
df.drop_duplicates(inplace=True)

# сплит

In [6]:
# сплит сгенерированной части на трейн и валид 
print(df["type"].unique())
traindf = df[~df["type"].isin(['students', 'crowdsource'])]

validIndex = set()
for c in traindf.columns:
    if c in ["x", "y_name", "type"]:
        continue
    train_x, valid_x = train_test_split(traindf.index, test_size=0.03, random_state=69, shuffle=True, stratify=traindf[c])
    validIndex |= set(valid_x)
print("valid percent", len(validIndex) / df.shape[0])
subsets = ["valid" if i in validIndex else "train" for i in range(df.shape[0])]
df["subset"] = subsets
df.loc[df["type"].isin(['students', 'crowdsource']),"subset"] = "test"

['generator' 'students' 'crowdsource']
valid percent 0.22293526785714285


In [7]:
for c in df.columns:
    if c in ["x", "y_name", "subset", "type"]:
        continue
    trains_counts = df.loc[df["subset"]=="train", c].value_counts().to_dict()
    valid_counts = df.loc[df["subset"]=="valid", c].value_counts().to_dict()
    test_counts = df.loc[df["subset"]=="test", c].value_counts().to_dict()
    print(c)
    for k in trains_counts:
        if k in test_counts:
            print(k, trains_counts[k], valid_counts[k], test_counts[k], valid_counts[k] / trains_counts[k], test_counts[k] / trains_counts[k])
        else:
            print(k, trains_counts[k], valid_counts[k], valid_counts[k] / trains_counts[k])
    print()

y
9 1676 303 0.1807875894988067
3 1675 351 0.20955223880597015
4 1573 403 0.256198347107438
2 1542 457 0.29636835278858625
11 1497 418 0.2792251169004676
7 1459 541 130 0.3708019191226868 0.08910212474297464
6 1365 485 0.3553113553113553
8 1317 683 995 0.5186028853454822 0.7555049354593774
10 1298 512 0.39445300462249616
5 1060 424 53 0.4 0.05
1 925 191 85 0.2064864864864865 0.0918918918918919
0 26 12 48 0.46153846153846156 1.8461538461538463

action
5 3248 754 0.23214285714285715
6 2809 1018 1178 0.3624065503737985 0.41936632253470985
8 1799 686 0.38132295719844356
9 1766 715 0.40486976217440546
0 1687 307 30 0.1819798458802608 0.01778304682868998
7 1622 644 0.3970406905055487
4 1542 457 0.29636835278858625
2 736 158 52 0.21467391304347827 0.07065217391304347
3 189 33 33 0.1746031746031746 0.1746031746031746
1 15 8 18 0.5333333333333333 1.2

direction
0 9698 3378 1226 0.34831924108063517 0.12641781810682615
3 1459 342 9 0.23440712816997944 0.00616860863605209
1 804 202 32 0.2512437810

In [8]:
df.to_csv("../../Data/Interim/24052022_onecmd_split.csv", index=False)
#df = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
#df.sample(2000).to_csv("../../Data/Interim/short_split.csv", index=False)

# folds

In [9]:
from sklearn.model_selection import StratifiedKFold
df = pd.read_csv("../../Data/Interim/24052022_onecmd_split.csv")
df_old_folds = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
df["fold"]=-1
df[df["subset"]=="test"]["y"].value_counts()

8    995
7    130
1     85
5     53
0     48
Name: y, dtype: int64

In [10]:
skf = StratifiedKFold(n_splits=5)

for fold_i, (train_index, test_index) in enumerate(skf.split(df[df["subset"]=="test"].index, df[df["subset"]=="test"]["y"])):
    df.loc[df[df["subset"]=="test"].index[test_index], "fold"] = fold_i
    print(len(train_index), len(test_index))
for i in range(5):
    print("fold#", i)
    print(df[df["fold"]==i]["y"].value_counts())

1048 263
1049 262
1049 262
1049 262
1049 262
fold# 0
8    199
7     26
1     17
5     11
0     10
Name: y, dtype: int64
fold# 1
8    199
7     26
1     17
0     10
5     10
Name: y, dtype: int64
fold# 2
8    199
7     26
1     17
0     10
5     10
Name: y, dtype: int64
fold# 3
8    199
7     26
1     17
5     11
0      9
Name: y, dtype: int64
fold# 4
8    199
7     26
1     17
5     11
0      9
Name: y, dtype: int64


In [11]:
df.to_csv("../../Data/Interim/24052022_onecmd_split.csv", index=False)